<a href="https://colab.research.google.com/github/Hubert26/suicides_IPPAN/blob/main/data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
import numpy as np
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
import os
from google.colab import files

#Ustawienie braku maksymalnej ilości wyświetlanych kolumn
pd.options.display.max_columns = None

# 2023

In [17]:
df_raw_2023 = pd.read_excel('https://raw.githubusercontent.com/Hubert26/suicides_IPPAN/main/data/Samobojstwa_2023.xlsx')

In [18]:
df_raw_2023.head(5)

,ID samobójcy,Data rejestracji,Przedział wiekowy,Płeć,Stan cywilny,Wykształcenie,Informacje o pracy i nauce,Źródło utrzymania,Czy samobójstwo zakończyło się zgonem,Miejsce zamachu,Klasa miejscowości,Sposób popełnienia,Powód zamachu *,Powód zamachu 2,Powód zamachu 3,Powód zamachu 4,Stan świadomości *,Informacje dotyczące stanu zdrowia *,Informacje dotyczące leczenia z powodu alkoholizmu/narkomanii,W ciągu ostatniego miesiąca sprawca zdarzenia miał przynajmniej jeden raz kontakt z *
0,138421942,2023-01-01 01:15:42,25-29,Mężczyzna,Konkubent/konkubina,Brak danych/nieustalone,Bezrobotny,Brak danych/nieustalony,N,Mieszkanie/dom,Wieś,Samookaleczenie powierzchowne,Inny niewymieniony powyżej,Nieporozumienie rodzinne/przemoc w rodzinie,NaN,NaN,Pod wpływem alkoholu,Brak danych/nieustalono,NaN,Brak możliwości ustalenia
1,138421944,2023-01-01 01:32:02,55-59,Mężczyzna,Żonaty/zamężna,Brak danych/nieustalone,Brak danych/nieustalono,Brak danych/nieustalony,N,Mieszkanie/dom,Miasto o liczbie ludności 20 000 – 49 999,Powieszenie się,Nieustalony,NaN,NaN,NaN,Brak danych/nieustalony,Brak danych/nieustalono,NaN,Brak możliwości ustalenia
2,138421949,2023-01-01 01:54:15,13-18,Kobieta,Kawaler/panna,Brak danych/nieustalone,Uczeń,Na utrzymaniu innej osoby,N,Mieszkanie/dom,Wieś,Zażycie innych leków,Nieporozumienie rodzinne/przemoc w rodzinie,NaN,NaN,NaN,Trzeźwy(a),Brak danych/nieustalono,NaN,Placówką leczniczą
3,138421953,2023-01-01 02:19:35,55-59,Mężczyzna,Żonaty/zamężna,Brak danych/nieustalone,Brak danych/nieustalono,Brak danych/nieustalony,N,Mieszkanie/dom,Miasto o liczbie ludności do 19 999,Samookaleczenie powierzchowne,Nieporozumienie rodzinne/przemoc w rodzinie,NaN,NaN,NaN,Pod wpływem alkoholu,Nadużywał(a) alkoholu,NaN,Brak możliwości ustalenia
4,138421954,2023-01-01 02:32:52,19-24,Kobieta,Konkubent/konkubina,Brak danych/nieustalone,Brak danych/nieustalono,Brak danych/nieustalony,N,Mieszkanie/dom,Miasto o liczbie ludności 100 000 – 499 999,Zażycie innych leków,Konflikt z osobami spoza rodziny,NaN,NaN,NaN,Pod wpływem alkoholu,Nadużywał(a) alkoholu,NaN,Brak możliwości ustalenia


##Zamiana nazw kolumn

In [19]:
# Zamiana spacji na podkreślenia w nazwach kolumn
df_raw_2023.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)


In [20]:
df_raw_2023.rename(columns={'Data_rejestracji': 'Date'}, inplace=True)

## Daty

In [21]:
# Wyodrębnienie roku i miesiąca
df_raw_2023['Date'] = pd.to_datetime(df_raw_2023['Date'])
df_raw_2023['DateY'] = df_raw_2023['Date'].dt.strftime('%Y')
df_raw_2023['DateM'] = df_raw_2023['Date'].dt.strftime('%m')

In [22]:
df_raw_2023['DateY'].unique()

array(['2023', nan], dtype=object)

In [23]:
df_raw_2023['DateM'].unique()

array(['01', '02', nan, '03', '04', '05', '06', '07', '08', '09', '10',
       '11', '12'], dtype=object)

In [24]:
# Połączenie roku i miesiąca w formacie MM.YYYY
df_raw_2023['Date'] = df_raw_2023['DateM'] + '.' + df_raw_2023['DateY']

In [25]:
df_raw_2023['Date'].unique()

array(['01.2023', '02.2023', nan, '03.2023', '04.2023', '05.2023',
       '06.2023', '07.2023', '08.2023', '09.2023', '10.2023', '11.2023',
       '12.2023'], dtype=object)

##Mapowanie

###Przedział_wiekowy

In [26]:
column = 'Przedział_wiekowy'
df_raw_2023.rename(columns={column: 'Age1'}, inplace=True)

In [29]:
df_raw_2023['Age1'].unique()

array(['25-29', '55-59', '13-18', '19-24', '30-34', '50-54', '40-44',
       '45-49', '35-39', '65-69', '75-79', '80-84', '60-64', '70-74',
       '85+', '07-12', nan], dtype=object)

In [28]:
column = 'Age1'
mapping = {
    "7-12": '07-12',
    '13-18': '13-18',
    '19-24': '19-24',
    '25-29': '25-29',
    '30-34': '30-34',
    '35-39': '35-39',
    '40-44': '40-44',
    '45-49': '45-49',
    '50-54': '50-54',
    '55-59': '55-59',
    '60-64': '60-64',
    '65-69': '65-69',
    '70-74': '70-74',
    '75-79': '75-79',
    '80-84': '80-84',
    '85+': '85+',
    'Nieustalony wiek': np.nan
}

df_raw_2023[column] = df_raw_2023[column].map(mapping)


In [30]:
df_raw_2023['Age2'] = df_raw_2023['Age1']

In [33]:
df_raw_2023['Age2'].unique()

array(['19-34', '35-64', '00-18', '65+', nan], dtype=object)

In [32]:
column = 'Age2'
mapping = {
    "07-12": '00-18',
    '13-18': '00-18',
    '19-24': '19-34',
    '25-29': '19-34',
    '30-34': '19-34',
    '35-39': '35-64',
    '40-44': '35-64',
    '45-49': '35-64',
    '50-54': '35-64',
    '55-59': '35-64',
    '60-64': '35-64',
    '65-69': '65+',
    '70-74': '65+',
    '75-79': '65+',
    '80-84': '65+',
    '85+': '65+'
}

df_raw_2023[column] = df_raw_2023[column].map(mapping)


###Płeć

In [34]:
df_raw_2023['Płeć'].unique()

array(['Mężczyzna', 'Kobieta', nan], dtype=object)

In [35]:
df_raw_2023.rename(columns={'Płeć': 'Gender'}, inplace=True)

In [36]:
column = 'Gender'
mapping = {
    "Kobieta": 'F',
    "Mężczyzna": 'M'
}

df_raw_2023[column] = df_raw_2023[column].map(mapping)


In [37]:
df_raw_2023['Gender'].unique()

array(['M', 'F', nan], dtype=object)

###Stan_cywilny

In [38]:
df_raw_2023['Stan_cywilny'].unique()

array(['Konkubent/konkubina', 'Żonaty/zamężna', 'Kawaler/panna',
       'Brak danych/nieustalony', 'Rozwiedziony/rozwiedziona',
       'Wdowiec/wdowa', 'Separowany/separowana', nan], dtype=object)

In [39]:
column = 'Stan_cywilny'
mapping = {
    "Brak danych/nieustalony": np.nan,
    "Kawaler/panna": 'Single ',
    "Konkubent/konkubina": 'Cohabitant',
    "Żonaty/zamężna": 'Married',
    "Separowany/separowana": 'Separated',
    "Rozwiedziony/rozwiedziona": 'Divorced',
    "Wdowiec/wdowa": 'Widowed'
}

df_raw_2023[column] = df_raw_2023[column].map(mapping)
df_raw_2023.rename(columns={column: 'Marital'}, inplace=True)


In [40]:
df_raw_2023['Marital'].unique()

array(['Cohabitant', 'Married', 'Single ', nan, 'Divorced', 'Widowed',
       'Separated'], dtype=object)

###Wykształcenie

In [41]:
df_raw_2023['Wykształcenie'].unique()

array(['Brak danych/nieustalone', 'Podstawowe', 'Średnie',
       'Zasadnicze zawodowe', 'Podstawowe niepełne', 'Wyższe',
       'Gimnazjalne', 'Policealne', nan], dtype=object)

In [42]:
column = 'Wykształcenie'
mapping = {
    'Brak danych/nieustalone': np.nan,
    'Podstawowe niepełne': 'Pre-primary',
    'Podstawowe': 'Primary',
    'Gimnazjalne': 'Secondary',
    'Zasadnicze zawodowe': 'Vocational',
    'Średnie': 'Secondary',
    'Policealne': 'Secondary',
    'Wyższe': 'Higher'
}

df_raw_2023[column] = df_raw_2023[column].map(mapping)
df_raw_2023.rename(columns={column: 'Education'}, inplace=True)



In [43]:
df_raw_2023['Education'].unique()

array([nan, 'Primary', 'Secondary', 'Vocational', 'Pre-primary', 'Higher'],
      dtype=object)

###Informacje_o_pracy_i_nauce

In [44]:
df_raw_2023['Informacje_o_pracy_i_nauce'].unique()

array(['Bezrobotny', 'Brak danych/nieustalono', 'Uczeń', 'Praca stała',
       'Rolnik', 'Praca dorywcza', 'Student',
       'Pracujący na własny rachunek/samodzielna działalność gospodarcza',
       nan], dtype=object)

In [45]:
column = 'Informacje_o_pracy_i_nauce'
mapping = {
    'Brak danych/nieustalono': np.nan,
    'Uczeń': 'Student',
    'Student': 'Student',
    'Rolnik': 'Agriculturalist',
    'Pracujący na własny rachunek/samodzielna działalność gospodarcza': 'Employed',
    'Praca stała': 'Employed',
    'Praca dorywcza': 'Employed',
    'Bezrobotny': 'Unemployed'
}

df_raw_2023[column] = df_raw_2023[column].map(mapping)
df_raw_2023.rename(columns={column: 'WorkInfo'}, inplace=True)

In [46]:
df_raw_2023['WorkInfo'].unique()

array(['Unemployed', nan, 'Student', 'Employed', 'Agriculturalist'],
      dtype=object)

###Źródło_utrzymania

In [47]:
df_raw_2023['Źródło_utrzymania'].unique()

array(['Brak danych/nieustalony', 'Na utrzymaniu innej osoby', 'Praca',
       'Emerytura', 'Nie ma stałego źródła utrzymania', 'Renta',
       'Zasiłek/alimenty', nan], dtype=object)

In [48]:
column = 'Źródło_utrzymania'
mapping = {
    'Brak danych/nieustalony': np.nan,
    'Na utrzymaniu innej osoby': 'Dependent',
    'Praca': 'Steady',
    'Emerytura': 'Benefits',
    'Renta': 'Benefits',
    'Zasiłek/alimenty': 'Benefits',
    'Nie ma stałego źródła utrzymania': 'NoSteady'
}

df_raw_2023[column] = df_raw_2023[column].map(mapping)
df_raw_2023.rename(columns={column: 'Income'}, inplace=True)

In [49]:
df_raw_2023['Income'].unique()

array([nan, 'Dependent', 'Steady', 'Benefits', 'NoSteady'], dtype=object)

###Czy_samobójstwo_zakończyło_się_zgonem

In [50]:
df_raw_2023['Czy_samobójstwo_zakończyło_się_zgonem'].unique()

array(['N', 'T', nan], dtype=object)

In [51]:
column = 'Czy_samobójstwo_zakończyło_się_zgonem'

mapping = {
    'T': 1,
    'N': 0
}
df_raw_2023[column] = df_raw_2023[column].map(mapping)
df_raw_2023.rename(columns={column: 'Fatal'}, inplace=True)

In [52]:
df_raw_2023['Fatal'].unique()

array([ 0.,  1., nan])

###Miejsce_zamachu

In [53]:
df_raw_2023['Miejsce_zamachu'].unique()

array(['Mieszkanie/dom', 'Rzeka/jezioro/inny zbiornik wodny',
       'Zabudowania gospodarcze', 'Garaż/piwnica/strych',
       'Droga/ulica/chodnik', 'Teren kolei/tory', 'Park, las', 'Inne',
       'Szkoła/uczelnia', 'Zakład pracy',
       'Placówka wychowawczo-opiekuńcza',
       'Placówka lecznicza lub sanatoryjna', 'Miejsce prawnej izolacji',
       'Obiekt policyjny', 'Obiekt wojskowy', nan], dtype=object)

In [54]:
column = 'Miejsce_zamachu'
mapping = {
    'Droga/ulica/chodnik': 'Road',
    'Zabudowania gospodarcze': 'UtilitySpaces',
    'Mieszkanie/dom': 'House',
    'Teren kolei/tory': 'Railway',
    'Park, las': 'Forest',
    'Garaż/piwnica/strych': 'House',
    'Rzeka/jezioro/inny zbiornik wodny': 'WaterRes',
    'Zakład pracy': 'Work',
    'Placówka lecznicza lub sanatoryjna': 'Institution',
    'Miejsce prawnej izolacji': 'Isolation',
    'Obiekt wojskowy': 'PoliceArmy',
    'Placówka wychowawczo-opiekuńcza': 'Institution',
    'Szkoła/uczelnia': 'School',
    'Obiekt policyjny': 'PoliceArmy',
    'Inne': 'Other'
}

df_raw_2023[column] = df_raw_2023[column].map(mapping)
df_raw_2023.rename(columns={column: 'Place'}, inplace=True)


In [55]:
df_raw_2023['Place'].unique()

array(['House', 'WaterRes', 'UtilitySpaces', 'Road', 'Railway', 'Forest',
       'Other', 'School', 'Work', 'Institution', 'Isolation',
       'PoliceArmy', nan], dtype=object)

###Sposób_popełnienia

In [56]:
df_raw_2023['Sposób_popełnienia'].unique()

array(['Samookaleczenie powierzchowne', 'Powieszenie się',
       'Zażycie innych leków',
       'Zażycie środków nasennych/leków psychotropowych',
       'Uszkodzenie układu krwionośnego', 'Inny',
       'Utonięcie/utopienie się', 'Rzucenie się pod pojazd w ruchu',
       'Uduszenie się', 'Rzucenie się z wysokości',
       'Zastrzelenie się/użycie broni palnej', 'Zatrucie gazem/spalinami',
       'Samopodpalenie', 'Zatrucie środkami chemicznymi/toksycznymi',
       'Zatrucie środkami odurzającymi', 'Zatrucie dopalaczami', nan],
      dtype=object)

In [57]:
column = 'Sposób_popełnienia'
mapping = {
    'Rzucenie się pod pojazd w ruchu': 'Vehicle',
    'Rzucenie się z wysokości': 'Jumping',
    'Powieszenie się': 'Hanging',
    'Uszkodzenie układu krwionośnego': 'Self-harm',
    'Zastrzelenie się/użycie broni palnej': 'Schooting',
    'Samookaleczenie powierzchowne': 'Self-harm',
    'Zażycie środków nasennych/leków psychotropowych': 'Drugs',
    'Zatrucie gazem/spalinami': 'Gas',
    'Zażycie innych leków': 'Drugs',
    'Zatrucie środkami chemicznymi/toksycznymi': 'Poisoning',
    'Zatrucie środkami odurzającymi': 'Drugs',
    'Zatrucie dopalaczami': 'Drugs',
    'Utonięcie/utopienie się': 'Drowning',
    'Samopodpalenie': 'Self-harm',
    'Uduszenie się': 'Other',
    'Inny': 'Other'
}

df_raw_2023[column] = df_raw_2023[column].map(mapping)
df_raw_2023.rename(columns={column: 'Method'}, inplace=True)

In [58]:
df_raw_2023['Method'].unique()

array(['Self-harm', 'Hanging', 'Drugs', 'Other', 'Drowning', 'Vehicle',
       'Jumping', 'Schooting', 'Gas', 'Poisoning', nan], dtype=object)

###Powód_zamachu

In [59]:
list(set(df_raw_2023['Powód_zamachu_*'].unique()) | set(df_raw_2023['Powód_zamachu_2'].unique()) | set(df_raw_2023['Powód_zamachu_3'].unique()) | set(df_raw_2023['Powód_zamachu_4'].unique()))

['Nieporozumienie rodzinne/przemoc w rodzinie',
 'Nagła utrata źródła utrzymania',
 'Zawód miłosny',
 'Leczony(a) psychiatrycznie',
 'Trwałe kalectwo',
 'Mobbing, cybermobbing, znęcanie',
 nan,
 'Konflikt z osobami spoza rodziny',
 'Nosiciel wirusa HIV, chory na AIDS',
 'Nieustalony',
 'Inny niewymieniony powyżej',
 'Problemy w szkole lub pracy',
 'Złe warunki ekonomiczne/długi',
 'Choroba fizyczna',
 'Dokonanie przestępstwa lub wykroczenia',
 'Pogorszenie lub nagła utrata zdrowia',
 'Choroba psychiczna/zaburzenia psychiczne',
 'Niepożądana ciąża',
 'Śmierć bliskiej osoby',
 'Zagrożenie lub utrata miejsca zamieszkania']

In [60]:
column = 'Powód_zamachu_*'
mapping = {
    'Nieustalony': np.nan,
    'Zawód miłosny': 'HeartBreak',
    'Leczony(a) psychiatrycznie': 'MentalHealth',
    'Nieporozumienie rodzinne/przemoc w rodzinie': 'FamilyConflict',
    'Nosiciel wirusa HIV, chory na AIDS': 'HealthLoss',
    'Nagła utrata źródła utrzymania': 'Finances',
    'Złe warunki ekonomiczne/długi': 'Finances',
    'Choroba psychiczna/zaburzenia psychiczne': 'MentalHealth',
    'Problemy w szkole lub pracy': 'SchoolWork',
    'Śmierć bliskiej osoby': 'CloseDeath',
    'Dokonanie przestępstwa lub wykroczenia': 'Crime',
    'Trwałe kalectwo': 'Disability',
    'Niepożądana ciąża': 'Other',
    'Choroba fizyczna': 'HealthLoss',
    'Pogorszenie lub nagła utrata zdrowia': 'HealthLoss',
    'Konflikt z osobami spoza rodziny': 'SchoolWork',
    'Zagrożenie lub utrata miejsca zamieszkania': 'Finances',
    'Mobbing, cybermobbing, znęcanie': 'SchoolWork',
    'Inny niewymieniony powyżej': 'Other'
}

df_raw_2023[column] = df_raw_2023[column].map(mapping)
df_raw_2023.rename(columns={column: 'Context'}, inplace=True)

###Powód_zamachu_2

In [61]:
column = 'Powód_zamachu_2'
mapping = {
    'Nieustalony': np.nan,
    'Zawód miłosny': 'HeartBreak',
    'Leczony(a) psychiatrycznie': 'MentalHealth',
    'Nieporozumienie rodzinne/przemoc w rodzinie': 'FamilyConflict',
    'Nosiciel wirusa HIV, chory na AIDS': 'HealthLoss',
    'Nagła utrata źródła utrzymania': 'Finances',
    'Złe warunki ekonomiczne/długi': 'Finances',
    'Choroba psychiczna/zaburzenia psychiczne': 'MentalHealth',
    'Problemy w szkole lub pracy': 'SchoolWork',
    'Śmierć bliskiej osoby': 'CloseDeath',
    'Dokonanie przestępstwa lub wykroczenia': 'Crime',
    'Trwałe kalectwo': 'Disability',
    'Niepożądana ciąża': 'Other',
    'Choroba fizyczna': 'HealthLoss',
    'Pogorszenie lub nagła utrata zdrowia': 'HealthLoss',
    'Konflikt z osobami spoza rodziny': 'SchoolWork',
    'Zagrożenie lub utrata miejsca zamieszkania': 'Finances',
    'Mobbing, cybermobbing, znęcanie': 'SchoolWork',
    'Inny niewymieniony powyżej': 'Other'
}

df_raw_2023[column] = df_raw_2023[column].map(mapping)
df_raw_2023.rename(columns={column: 'Context2'}, inplace=True)

###Powód_zamachu_3

In [62]:
column = 'Powód_zamachu_3'
mapping = {
    'Nieustalony': np.nan,
    'Zawód miłosny': 'HeartBreak',
    'Leczony(a) psychiatrycznie': 'MentalHealth',
    'Nieporozumienie rodzinne/przemoc w rodzinie': 'FamilyConflict',
    'Nosiciel wirusa HIV, chory na AIDS': 'HealthLoss',
    'Nagła utrata źródła utrzymania': 'Finances',
    'Złe warunki ekonomiczne/długi': 'Finances',
    'Choroba psychiczna/zaburzenia psychiczne': 'MentalHealth',
    'Problemy w szkole lub pracy': 'SchoolWork',
    'Śmierć bliskiej osoby': 'CloseDeath',
    'Dokonanie przestępstwa lub wykroczenia': 'Crime',
    'Trwałe kalectwo': 'Disability',
    'Niepożądana ciąża': 'Other',
    'Choroba fizyczna': 'HealthLoss',
    'Pogorszenie lub nagła utrata zdrowia': 'HealthLoss',
    'Konflikt z osobami spoza rodziny': 'SchoolWork',
    'Zagrożenie lub utrata miejsca zamieszkania': 'Finances',
    'Mobbing, cybermobbing, znęcanie': 'SchoolWork',
    'Inny niewymieniony powyżej': 'Other'
}

df_raw_2023[column] = df_raw_2023[column].map(mapping)
df_raw_2023.rename(columns={column: 'Context3'}, inplace=True)

###Powód_zamachu_4

In [63]:
column = 'Powód_zamachu_4'
mapping = {
    'Nieustalony': np.nan,
    'Zawód miłosny': 'HeartBreak',
    'Leczony(a) psychiatrycznie': 'MentalHealth',
    'Nieporozumienie rodzinne/przemoc w rodzinie': 'FamilyConflict',
    'Nosiciel wirusa HIV, chory na AIDS': 'HealthLoss',
    'Nagła utrata źródła utrzymania': 'Finances',
    'Złe warunki ekonomiczne/długi': 'Finances',
    'Choroba psychiczna/zaburzenia psychiczne': 'MentalHealth',
    'Problemy w szkole lub pracy': 'SchoolWork',
    'Śmierć bliskiej osoby': 'CloseDeath',
    'Dokonanie przestępstwa lub wykroczenia': 'Crime',
    'Trwałe kalectwo': 'Disability',
    'Niepożądana ciąża': 'Other',
    'Choroba fizyczna': 'HealthLoss',
    'Pogorszenie lub nagła utrata zdrowia': 'HealthLoss',
    'Konflikt z osobami spoza rodziny': 'SchoolWork',
    'Zagrożenie lub utrata miejsca zamieszkania': 'Finances',
    'Mobbing, cybermobbing, znęcanie': 'SchoolWork',
    'Inny niewymieniony powyżej': 'Other'
}

df_raw_2023[column] = df_raw_2023[column].map(mapping)
df_raw_2023.rename(columns={column: 'Context4'}, inplace=True)

In [64]:
list(set(df_raw_2023['Context'].unique()) | set(df_raw_2023['Context2'].unique()) | set(df_raw_2023['Context3'].unique()) | set(df_raw_2023['Context4'].unique()))

[nan,
 'FamilyConflict',
 'Crime',
 'Disability',
 'Finances',
 'HeartBreak',
 'Other',
 'MentalHealth',
 'HealthLoss',
 'CloseDeath',
 'SchoolWork']

###Stan_świadomości

In [65]:
df_raw_2023['Stan_świadomości_*'].unique()

array(['Pod wpływem alkoholu', 'Brak danych/nieustalony', 'Trzeźwy(a)',
       'Pod wpływem leków', 'Pod wpływem alkoholu i środków odurzających',
       'Pod wpływem alkoholu i leków', 'Pod wpływem środków odurzających',
       'Pod wpływem alkoholu, leków i środków odurzających',
       'Pod wpływem zastępczych środków/substancji (dopalaczy)',
       'Pod wpływem alkoholu i zastępczych środków/substancji (dopalaczy)',
       'Pod wpływem leków i środków odurzających',
       'Pod wpływem alkoholu zastępczych środków/substancji (dopalaczy)',
       'Pod wpływem leków i srodków odurzających', nan], dtype=object)

In [66]:
column = 'Stan_świadomości_*'
mapping = {
    'Brak danych/nieustalony': np.nan,
    'Trzeźwy(a)': 'Sober',
    'Pod wpływem alkoholu': 'Alco',
    'Pod wpływem zastępczych środków/substancji (dopalaczy)': 'OtherSub',
    'Pod wpływem leków': 'OtherSub',
    'Pod wpływem środków odurzających': 'OtherSub',
    'Pod wpływem alkoholu i zastępczych środków/substancji (dopalaczy)': 'Alco&OtherSub',
    'Pod wpływem alkoholu zastępczych środków/substancji (dopalaczy)': 'Alco&OtherSub',
    'Pod wpływem alkoholu i leków': 'Alco&OtherSub',
    'Pod wpływem alkoholu i środków odurzających': 'Alco&OtherSub',
    'Pod wpływem leków i środków odurzających': 'OtherSub',
    'Pod wpływem alkoholu, leków i środków odurzających': 'Alco&OtherSub'
}

df_raw_2023[column] = df_raw_2023[column].map(mapping)
df_raw_2023.rename(columns={column: 'Substance'}, inplace=True)

In [67]:
df_raw_2023['Substance'].unique()

array(['Alco', nan, 'Sober', 'OtherSub', 'Alco&OtherSub'], dtype=object)

###Informacje_dotyczące_leczenia_z_powodu_alkoholizmu/narkomanii

In [68]:
df_raw_2023['Informacje_dotyczące_leczenia_z_powodu_alkoholizmu/narkomanii'].unique()

array([nan, 'Nadużywał(a) alkoholu', 'Leczony(a) z powodu alkoholizmu',
       'Leczony(a) z powodu narkomanii',
       'Leczony(a) z powodu alkoholizmu i narkomanii',
       'Leczony(a) psychiatrycznie'], dtype=object)

In [69]:
column = 'Informacje_dotyczące_leczenia_z_powodu_alkoholizmu/narkomanii'
mapping = {
    'Leczony(a) psychiatrycznie': np.nan,
    'Nadużywał(a) alkoholu': 'Alco',
    'Leczony(a) z powodu alkoholizmu': 'Alco',
    'Leczony(a) z powodu narkomanii': 'OtherSub',
    'Leczony(a) z powodu alkoholizmu i narkomanii': 'Alco&OtherSub'
}

df_raw_2023[column] = df_raw_2023[column].map(mapping)
df_raw_2023.rename(columns={column: 'AbuseInfo'}, inplace=True)

In [70]:
df_raw_2023['AbuseInfo'].unique()

array([nan, 'Alco', 'OtherSub', 'Alco&OtherSub'], dtype=object)

In [81]:
df_raw_2023['Informacje_dotyczące_stanu_zdrowia_*'].unique()

array([nan, 'Alco', 'Alco&OtherSub', 'OtherSub'], dtype=object)

In [80]:
column = 'Informacje_dotyczące_stanu_zdrowia_*'
mapping = {
    'Brak danych/nieustalono': np.nan,
    'Nadużywał(a) alkoholu': 'Alco',
    'Leczony(a) psychiatrycznie': np.nan,
    'Leczony(a) z powodu alkoholizmu': 'Alco',
    'Choroba fizyczna': np.nan,
    'Trwałe kalectwo': np.nan,
    'Zatrzymany(a) w izbie wytrzeźwień': 'Alco',
    'Nadużywał(a) alkoholu i narkotyków': 'Alco&OtherSub',
    'Nadużywał(a) alkoholu i nakrotyków': 'Alco&OtherSub',
    'Leczony(a) z powodu narkomanii': 'OtherSub',
    'Używał dopalaczy i narkotyków': 'OtherSub',
    'Nadużywał(a) alkoholu i narkotykó': 'OtherSub',
    'Nadużywał(a) alkoholu, dopalaczy i narkotyków': 'Alco&OtherSub',
    'Nadużywał(a) alkoholu, narkotyków i dopalaczy': 'Alco&OtherSub',
    'Nadużywał(a) alkoholu i dopalaczy': 'Alco&OtherSub',
    'Używał dopalaczy': 'OtherSub',
    'Nadużywał(a) alkoholu, dopalaczy, narkotyków': 'Alco&OtherSub',
    'Leczony(a) psychiatrycznie, nadużywał(a) alkoholu': 'Alco',
}

df_raw_2023[column] = df_raw_2023[column].map(mapping)

In [82]:
df_raw_2023['AbuseInfo'] = df_raw_2023['AbuseInfo'].fillna(df_raw_2023['Informacje_dotyczące_stanu_zdrowia_*'])

In [83]:
df_raw_2023.drop(columns='Informacje_dotyczące_stanu_zdrowia_*', inplace=True)

##Powód zamachu - dummies

In [71]:
columns_to_drop = ['Context', 'Context2', 'Context3', 'Context4']
df_powód_zamachu_2023 = df_raw_2023[columns_to_drop]
df_raw_2023.drop(columns=columns_to_drop, inplace=True)


In [72]:
powód_zamachu_values = (
    set(df_powód_zamachu_2023['Context'].unique()) |
    set(df_powód_zamachu_2023['Context2'].unique()) |
    set(df_powód_zamachu_2023['Context3'].unique()) |
    set(df_powód_zamachu_2023['Context4'].unique())
)

In [73]:
powód_zamachu_values

{'CloseDeath',
 'Crime',
 'Disability',
 'FamilyConflict',
 'Finances',
 'HealthLoss',
 'HeartBreak',
 'MentalHealth',
 'Other',
 'SchoolWork',
 nan}

In [74]:
for value in powód_zamachu_values:
    column_name = 'Context_' + str(value)
    df_powód_zamachu_2023[column_name] = df_powód_zamachu_2023.apply(lambda row: 1 if value in row.values else 0, axis=1)


<ipython-input-74-2ae7b8e01b52>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_powód_zamachu_2023[column_name] = df_powód_zamachu_2023.apply(lambda row: 1 if value in row.values else 0, axis=1)
<ipython-input-74-2ae7b8e01b52>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_powód_zamachu_2023[column_name] = df_powód_zamachu_2023.apply(lambda row: 1 if value in row.values else 0, axis=1)
<ipython-input-74-2ae7b8e01b52>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

In [75]:
columns_to_drop = ['Context', 'Context2', 'Context3', 'Context4', 'Context_nan']
df_powód_zamachu_2023.drop(columns=columns_to_drop, inplace=True)


<ipython-input-75-1efdf2541b49>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_powód_zamachu_2023.drop(columns=columns_to_drop, inplace=True)


In [76]:
df_powód_zamachu_2023.head(5)

,Context_FamilyConflict,Context_Crime,Context_Disability,Context_Finances,Context_HeartBreak,Context_Other,Context_MentalHealth,Context_HealthLoss,Context_CloseDeath,Context_SchoolWork
0,1,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1


##Liczba_powodów_zamachu

In [77]:
df_raw_2023['CountContext'] = df_powód_zamachu_2023.sum(axis=1)

In [78]:
df_raw_2023.head(5)

,ID_samobójcy,Date,Age1,Gender,Marital,Education,WorkInfo,Income,Fatal,Place,Klasa_miejscowości,Method,Substance,Informacje_dotyczące_stanu_zdrowia_*,AbuseInfo,W_ciągu_ostatniego_miesiąca_sprawca_zdarzenia_miał_przynajmniej_jeden_raz_kontakt_z_*,DateY,DateM,Age2,CountContext
0,138421942,01.2023,25-29,M,Cohabitant,NaN,Unemployed,NaN,0.0,House,Wieś,Self-harm,Alco,Brak danych/nieustalono,NaN,Brak możliwości ustalenia,2023,01,19-34,2
1,138421944,01.2023,55-59,M,Married,NaN,NaN,NaN,0.0,House,Miasto o liczbie ludności 20 000 – 49 999,Hanging,NaN,Brak danych/nieustalono,NaN,Brak możliwości ustalenia,2023,01,35-64,0
2,138421949,01.2023,13-18,F,Single,NaN,Student,Dependent,0.0,House,Wieś,Drugs,Sober,Brak danych/nieustalono,NaN,Placówką leczniczą,2023,01,00-18,1
3,138421953,01.2023,55-59,M,Married,NaN,NaN,NaN,0.0,House,Miasto o liczbie ludności do 19 999,Self-harm,Alco,Nadużywał(a) alkoholu,NaN,Brak możliwości ustalenia,2023,01,35-64,1
4,138421954,01.2023,19-24,F,Cohabitant,NaN,NaN,NaN,0.0,House,Miasto o liczbie ludności 100 000 – 499 999,Drugs,Alco,Nadużywał(a) alkoholu,NaN,Brak możliwości ustalenia,2023,01,19-34,1


# 2013_2022

In [ ]:
df_raw_2013_2022 = pd.read_csv('https://raw.githubusercontent.com/Hubert26/suicides_IPPAN/main/data/final_samobojstwa_2013_2022.csv', delimiter=',', low_memory=False)

In [ ]:
df_raw_2013_2022.head(5)

,ID samobójcy,Data raportu [RRRRMM],Przedział wiekowy,Płeć,Stan cywilny,Wykształcenie,Informacje o pracy i nauce,Źródło utrzymania,Czy samobójstwo zakończyło się zgonem,Miejsce zamachu,Sposób popełnienia,Powód zamachu,Powód zamachu 2,Powód zamachu 3,Powód zamachu 4,Stan świadomości,Informacje o używaniu substancji,Informacje dotyczące leczenia z powodu alkoholizmu/narkomanii
0,"114581552,00",201301,0,2,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0,0,0
1,"114584284,00",201301,2,2,0,0,1,0,1,1,1,0,NaN,NaN,NaN,0,0,0
2,"114584293,00",201301,2,1,1,2,1,1,1,15,2,0,NaN,NaN,NaN,0,0,0
3,"114584312,00",201301,3,2,1,0,0,0,1,0,18,1,NaN,NaN,NaN,0,0,0
4,"114584399,00",201301,7,2,1,0,9,0,1,2,3,0,NaN,NaN,NaN,2,0,0


##Zamiana nazw kolumn

In [ ]:
# Zamiana spacji na podkreślenia w nazwach kolumn
df_raw_2013_2022.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)


In [ ]:
df_raw_2013_2022.rename(columns={'Data_raportu_[RRRRMM]': 'Data'}, inplace=True)

## Daty

In [ ]:
# Wyodrębnienie roku i miesiąca
df_raw_2013_2022['Data_rok'] = df_raw_2013_2022['Data'].apply(lambda x: x[0:4] if len(x) > 8 else x[:4])
df_raw_2013_2022['Data_miesiąc'] = df_raw_2013_2022['Data'].apply(lambda x: x[5:7] if len(x) > 8 else x[4:6])

In [ ]:
df_raw_2013_2022['Data_rok'].unique()

array(['2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020',
       '2021', '2022'], dtype=object)

In [ ]:
df_raw_2013_2022['Data_miesiąc'].unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12'], dtype=object)

In [ ]:
# Połączenie roku i miesiąca w formacie MM.YYYY
df_raw_2013_2022['Data'] = df_raw_2013_2022['Data_miesiąc'] + '.' + df_raw_2013_2022['Data_rok']

In [ ]:
df_raw_2013_2022['Data'].unique()

array(['01.2013', '02.2013', '03.2013', '04.2013', '05.2013', '06.2013',
       '07.2013', '08.2013', '09.2013', '10.2013', '11.2013', '12.2013',
       '01.2014', '02.2014', '03.2014', '04.2014', '05.2014', '06.2014',
       '07.2014', '08.2014', '09.2014', '10.2014', '11.2014', '12.2014',
       '01.2015', '02.2015', '03.2015', '04.2015', '05.2015', '06.2015',
       '07.2015', '08.2015', '09.2015', '10.2015', '11.2015', '12.2015',
       '01.2016', '02.2016', '03.2016', '04.2016', '05.2016', '06.2016',
       '07.2016', '08.2016', '09.2016', '10.2016', '11.2016', '12.2016',
       '01.2017', '02.2017', '03.2017', '04.2017', '05.2017', '06.2017',
       '07.2017', '08.2017', '09.2017', '10.2017', '11.2017', '12.2017',
       '01.2018', '02.2018', '03.2018', '04.2018', '05.2018', '06.2018',
       '07.2018', '08.2018', '09.2018', '10.2018', '11.2018', '12.2018',
       '01.2019', '02.2019', '03.2019', '04.2019', '05.2019', '06.2019',
       '07.2019', '08.2019', '09.2019', '10.2019', 

##Mapowanie

###Przedział_wiekowy

In [ ]:
column = 'Przedział_wiekowy'
mapping = {
    0: np.nan,
    1: '07-12',
    2: '13-18',
    3: '19-24',
    4: '25-29',
    5: '30-34',
    6: '35-39',
    7: '40-44',
    8: '45-49',
    9: '50-54',
    10: '55-59',
    11: '60-64',
    12: '65-69',
    13: '70-74',
    14: '75-79',
    15: '80-84',
    16: '85+'
}

df_raw_2013_2022[column] = df_raw_2013_2022[column].map(mapping)
df_raw_2013_2022.rename(columns={column: 'Wiek'}, inplace=True)

###Płeć

In [ ]:
column = 'Płeć'
mapping = {
    0: np.nan,
    1: 'K',
    2: 'M'
}

df_raw_2013_2022[column] = df_raw_2013_2022[column].map(mapping)


###Stan_cywilny

In [ ]:
column = 'Stan_cywilny'
mapping = {
    0: np.nan,
    1: 'Samotność',
    2: 'Partnerstwo',
    3: 'Małżeństwo',
    4: 'Separacja',
    5: 'Rozwód',
    6: 'Wdowieństwo'
}

df_raw_2013_2022[column] = df_raw_2013_2022[column].map(mapping)
df_raw_2013_2022.rename(columns={column: 'St_cyw'}, inplace=True)


###Wykształcenie

In [ ]:
column = 'Wykształcenie'
mapping = {
    0: np.nan,
    1: 'Podst_niepełne',
    2: 'Podst',
    3: 'Gimnazjalne',
    4: 'Zawodowe',
    5: 'Średnie',
    6: 'Policealne',
    7: 'Wyższe'
}

df_raw_2013_2022[column] = df_raw_2013_2022[column].map(mapping)
df_raw_2013_2022.rename(columns={column: 'Poz_edu'}, inplace=True)



###Informacje_o_pracy_i_nauce

In [ ]:
column = 'Informacje_o_pracy_i_nauce'
mapping = {
    0: np.nan,
    1: 'Uczeń',
    2: 'Student',
    3: 'Umysłowa',
    4: 'Fizyczna',
    5: 'Rolnik',
    6: 'Własna',
    7: 'Stała',
    8: 'Dorywcza',
    9: 'Bezrobotny'
}

df_raw_2013_2022[column] = df_raw_2013_2022[column].map(mapping)
df_raw_2013_2022.rename(columns={column: 'Inf_praca'}, inplace=True)

###Źródło_utrzymania

In [ ]:
column = 'Źródło_utrzymania'
mapping = {
    0: np.nan,  # Zamiana 0 na NaN
    1: 'Zależny',
    2: 'Praca',
    3: 'Świadczenia',
    4: 'Bez_dochodu'
}

df_raw_2013_2022[column] = df_raw_2013_2022[column].map(mapping)
df_raw_2013_2022.rename(columns={column: 'Utrzymanie'}, inplace=True)

###Czy_samobójstwo_zakończyło_się_zgonem

In [ ]:
column = 'Czy_samobójstwo_zakończyło_się_zgonem'

mapping = {
    0: np.nan,  # Zamiana 0 na NaN
    1: 1,
    2: 0
}
df_raw_2013_2022[column] = df_raw_2013_2022[column].map(mapping)
df_raw_2013_2022.rename(columns={column: 'Sukces'}, inplace=True)

###Miejsce_zamachu

In [ ]:
column = 'Miejsce_zamachu'
mapping = {
    0: np.nan,
    1: 'Droga',
    2: 'Zabudowania',
    3: 'Mieszkanie',
    4: 'Tory',
    5: 'Park',
    6: 'Piwnica',
    7: 'Woda',
    8: 'Praca',
    9: 'Lecznica',
    10: 'Izolacja',
    11: 'Wojsko',
    12: 'Opieka',
    13: 'Szkoła',
    14: 'Policja',
    15: 'Inne'
}

df_raw_2013_2022[column] = df_raw_2013_2022[column].map(mapping)
df_raw_2013_2022.rename(columns={column: 'Miejsce'}, inplace=True)


###Sposób_popełnienia

In [ ]:
column = 'Sposób_popełnienia'
mapping = {
    0: np.nan,
    1: 'Pod_pojazd',
    2: 'Z_wysokości',
    3: 'Powieszenie',
    4: 'Układ_krwionośny',
    5: 'Zastrzelenie',
    6: 'Samookaleczenie',
    7: 'Samookaleczenie',
    8: 'Środki_nasenne',
    9: 'Trucizna',
    10: 'Gaz',
    11: 'Inne_leki',
    12: 'Środki_chemiczne',
    13: 'Odurzające',
    14: 'Dopalacze',
    15: 'Utonięcie',
    16: 'Podpalenie',
    17: 'Uduszenie',
    18: 'Inny'
}

df_raw_2013_2022[column] = df_raw_2013_2022[column].map(mapping)
df_raw_2013_2022.rename(columns={column: 'Sposób'}, inplace=True)

###Powód_zamachu

In [ ]:
column = 'Powód_zamachu'
mapping = {
    0: np.nan,
    1: 'Miłość',
    2: 'Choroby_psych',
    3: 'Konflikty_rodzinne',
    4: 'HIV/AIDS',
    5: 'Utrata_utrzymania',
    6: 'Finanse',
    7: 'Choroba',
    8: 'Praca',
    9: 'Śmierć_bliskiego',
    10: 'Przestępstwa',
    11: 'Kalectwo',
    12: 'Ciąża',
    13: 'Choroby_fiz',
    14: 'Zdrowie',
    15: 'Konflikty_inni',
    16: 'Bezdomność',
    17: 'Mobbing',
    18: 'Inne'
}

df_raw_2013_2022[column] = df_raw_2013_2022[column].map(mapping)
df_raw_2013_2022.rename(columns={column: 'Powód'}, inplace=True)

###Powód_zamachu_2

In [ ]:
column = 'Powód_zamachu_2'
mapping = {
    0: np.nan,
    1: 'Miłość',
    2: 'Choroby_psych',
    3: 'Konflikty_rodzinne',
    4: 'HIV/AIDS',
    5: 'Utrata_utrzymania',
    6: 'Finanse',
    7: 'Choroba',
    8: 'Praca',
    9: 'Śmierć_bliskiego',
    10: 'Przestępstwa',
    11: 'Kalectwo',
    12: 'Ciąża',
    13: 'Choroby_fiz',
    14: 'Zdrowie',
    15: 'Konflikty_inni',
    16: 'Bezdomność',
    17: 'Mobbing',
    18: 'Inne'
}

df_raw_2013_2022[column] = df_raw_2013_2022[column].map(mapping)
df_raw_2013_2022.rename(columns={column: 'Powód_2'}, inplace=True)

###Powód_zamachu_3

In [ ]:
column = 'Powód_zamachu_3'
mapping = {
    0: np.nan,
    1: 'Miłość',
    2: 'Choroby_psych',
    3: 'Konflikty_rodzinne',
    4: 'HIV/AIDS',
    5: 'Utrata_utrzymania',
    6: 'Finanse',
    7: 'Choroba',
    8: 'Praca',
    9: 'Śmierć_bliskiego',
    10: 'Przestępstwa',
    11: 'Kalectwo',
    12: 'Ciąża',
    13: 'Choroby_fiz',
    14: 'Zdrowie',
    15: 'Konflikty_inni',
    16: 'Bezdomność',
    17: 'Mobbing',
    18: 'Inne'
}

df_raw_2013_2022[column] = df_raw_2013_2022[column].map(mapping)
df_raw_2013_2022.rename(columns={column: 'Powód_3'}, inplace=True)

###Powód_zamachu_4

In [ ]:
column = 'Powód_zamachu_4'
mapping = {
    0: np.nan,
    1: 'Miłość',
    2: 'Choroby_psych',
    3: 'Konflikty_rodzinne',
    4: 'HIV/AIDS',
    5: 'Utrata_utrzymania',
    6: 'Finanse',
    7: 'Choroba',
    8: 'Praca',
    9: 'Śmierć_bliskiego',
    10: 'Przestępstwa',
    11: 'Kalectwo',
    12: 'Ciąża',
    13: 'Choroby_fiz',
    14: 'Zdrowie',
    15: 'Konflikty_inni',
    16: 'Bezdomność',
    17: 'Mobbing',
    18: 'Inne'
}

df_raw_2013_2022[column] = df_raw_2013_2022[column].map(mapping)
df_raw_2013_2022.rename(columns={column: 'Powód_4'}, inplace=True)

###Stan_świadomości

In [ ]:
column = 'Stan_świadomości'
mapping = {
    0: np.nan,
    1: 'Trzeźwy',
    2: 'Alko',
    3: 'Dopa',
    4: 'Narko',
    5: 'Leki',
    6: 'Odurza',
    7: 'Alko+dopa',
    8: 'Alko+narko',
    9: 'Alko+leki',
    10: 'Alko+odurza',
    11: 'Alko+dopa+odurza',
    12: 'Dopa+odurza',
    13: 'Leki+odurza',
    14: 'Leki+dopa',
    15: 'Leki+narko',
    16: 'Leki+dopa+odurza',
    17: 'Alko+leki+odurza',
    18: 'Alko+leki+dopa+odurza'
}

df_raw_2013_2022[column] = df_raw_2013_2022[column].map(mapping)
df_raw_2013_2022.rename(columns={column: 'Stan'}, inplace=True)

###Informacje_o_używaniu_substancji

In [ ]:
column = 'Informacje_o_używaniu_substancji'
mapping = {
    0: np.nan,
    1: 'Alko',
    2: 'Narko',
    3: 'Dopa',
    4: 'Alko+dopa',
    5: 'Narko+dopa',
    6: 'Alko+narko',
    7: 'Alko+narko+dopa'
}

df_raw_2013_2022[column] = df_raw_2013_2022[column].map(mapping)
df_raw_2013_2022.rename(columns={column: 'Używanie'}, inplace=True)


###Informacje_dotyczące_leczenia_z_powodu_alkoholizmu/narkomanii

In [ ]:
column = 'Informacje_dotyczące_leczenia_z_powodu_alkoholizmu/narkomanii'
mapping = {
    0: np.nan,
    1: 'Alko',
    2: 'Narko',
    3: 'Alko+narko'
}

df_raw_2013_2022[column] = df_raw_2013_2022[column].map(mapping)
df_raw_2013_2022.rename(columns={column: 'Leczenie'}, inplace=True)

##Powód zamachu - dummies

In [ ]:
columns_to_drop = ['Powód', 'Powód_2', 'Powód_3', 'Powód_4']
df_powód_zamachu_2013_2022 = df_raw_2013_2022[columns_to_drop]
df_raw_2013_2022.drop(columns=columns_to_drop, inplace=True)


In [ ]:
powód_zamachu_values = (
    set(df_powód_zamachu_2013_2022['Powód'].unique()) |
    set(df_powód_zamachu_2013_2022['Powód_2'].unique()) |
    set(df_powód_zamachu_2013_2022['Powód_3'].unique()) |
    set(df_powód_zamachu_2013_2022['Powód_4'].unique())
)

In [ ]:
powód_zamachu_values

{'Bezdomność',
 'Choroba',
 'Choroby_fiz',
 'Choroby_psych',
 'Ciąża',
 'Finanse',
 'HIV/AIDS',
 'Inne',
 'Kalectwo',
 'Konflikty_inni',
 'Konflikty_rodzinne',
 'Miłość',
 'Mobbing',
 'Praca',
 'Przestępstwa',
 'Utrata_utrzymania',
 'Zdrowie',
 nan,
 'Śmierć_bliskiego'}

In [ ]:
for value in powód_zamachu_values:
    column_name = 'Powód_' + str(value)
    df_powód_zamachu_2013_2022[column_name] = df_powód_zamachu_2013_2022.apply(lambda row: 1 if value in row.values else 0, axis=1)


<ipython-input-192-6e34b0c696c3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_powód_zamachu_2013_2022[column_name] = df_powód_zamachu_2013_2022.apply(lambda row: 1 if value in row.values else 0, axis=1)
<ipython-input-192-6e34b0c696c3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_powód_zamachu_2013_2022[column_name] = df_powód_zamachu_2013_2022.apply(lambda row: 1 if value in row.values else 0, axis=1)
<ipython-input-192-6e34b0c696c3>:3: SettingWithCopyWarning: 
A value is trying to be set on 

In [ ]:
columns_to_drop = ['Powód', 'Powód_2', 'Powód_3', 'Powód_4', 'Powód_nan']
df_powód_zamachu_2013_2022.drop(columns=columns_to_drop, inplace=True)


##Liczba_powodów_zamachu

In [ ]:
df_raw_2013_2022['L_powodów'] = df_powód_zamachu_2013_2022.sum(axis=1)

In [ ]:
df_raw_2013_2022.head(5)

,ID_samobójcy,Data,Wiek,Płeć,St_cyw,Poz_edu,Inf_praca,Utrzymanie,Sukces,Miejsce,Sposób,Stan,Używanie,Leczenie,Data_rok,Data_miesiąc,L_powodów
0,"114581552,00",01.2013,NaN,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,01,0
1,"114584284,00",01.2013,13-18,M,NaN,NaN,Uczeń,NaN,1.0,Droga,Pod_pojazd,NaN,NaN,NaN,2013,01,0
2,"114584293,00",01.2013,13-18,K,Samotność,Podst,Uczeń,Zależny,1.0,Inne,Z_wysokości,NaN,NaN,NaN,2013,01,0
3,"114584312,00",01.2013,19-24,M,Samotność,NaN,NaN,NaN,1.0,NaN,Inny,NaN,NaN,NaN,2013,01,1
4,"114584399,00",01.2013,40-44,M,Samotność,NaN,Bezrobotny,NaN,1.0,Zabudowania,Powieszenie,Alko,NaN,NaN,2013,01,0


#Scalenie

##df_raw

In [ ]:
# Znajdź wspólne kolumny
wspolne_kolumny = list(set(df_raw_2023.columns) & set(df_raw_2013_2022.columns))

# Wybierz wspólne kolumny z obu DataFrame'ów
df_raw_2023_selected = df_raw_2023[wspolne_kolumny]
df_raw_2013_2022_selected = df_raw_2013_2022[wspolne_kolumny]

# Połącz oba DataFrame'y
df_raw = pd.concat([df_raw_2013_2022_selected, df_raw_2023_selected])


In [ ]:
df_raw.head(1)

,Stan,Data_rok,Data_miesiąc,Utrzymanie,St_cyw,ID_samobójcy,Wiek,Miejsce,Leczenie,Sposób,Data,Poz_edu,Inf_praca,Płeć,L_powodów,Sukces
0,NaN,2013,01,NaN,NaN,"114581552,00",NaN,NaN,NaN,NaN,01.2013,NaN,NaN,M,0,NaN


In [ ]:
df_raw.shape

(128332, 16)

In [ ]:
df_raw_2013_2022.head(1)

,ID_samobójcy,Data,Wiek,Płeć,St_cyw,Poz_edu,Inf_praca,Utrzymanie,Sukces,Miejsce,Sposób,Stan,Używanie,Leczenie,Data_rok,Data_miesiąc,L_powodów
0,"114581552,00",01.2013,NaN,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,01,0


In [ ]:
df_raw_2013_2022.shape

(113196, 17)

In [ ]:
df_raw_2023.head(1)

,ID_samobójcy,Data,Wiek,Płeć,St_cyw,Poz_edu,Inf_praca,Utrzymanie,Sukces,Miejsce,Klasa_miejscowości,Sposób,Stan,Informacje_dotyczące_stanu_zdrowia_*,Leczenie,W_ciągu_ostatniego_miesiąca_sprawca_zdarzenia_miał_przynajmniej_jeden_raz_kontakt_z_*,Data_rok,Data_miesiąc,L_powodów
0,138421942,01.2023,25-29,M,Partnerstwo,NaN,Bezrobotny,NaN,0.0,Mieszkanie,Wieś,Samookaleczenie,Alko,Brak danych/nieustalono,NaN,Brak możliwości ustalenia,2023,01,2


In [ ]:
df_raw_2023.shape

(15136, 19)

In [ ]:
df_raw['Data_rok'].unique()

array(['2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020',
       '2021', '2022', '2023', nan], dtype=object)

In [ ]:
df_raw['Data_miesiąc'].unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', nan], dtype=object)

In [ ]:
df_raw['Data'].unique()

array(['01.2013', '02.2013', '03.2013', '04.2013', '05.2013', '06.2013',
       '07.2013', '08.2013', '09.2013', '10.2013', '11.2013', '12.2013',
       '01.2014', '02.2014', '03.2014', '04.2014', '05.2014', '06.2014',
       '07.2014', '08.2014', '09.2014', '10.2014', '11.2014', '12.2014',
       '01.2015', '02.2015', '03.2015', '04.2015', '05.2015', '06.2015',
       '07.2015', '08.2015', '09.2015', '10.2015', '11.2015', '12.2015',
       '01.2016', '02.2016', '03.2016', '04.2016', '05.2016', '06.2016',
       '07.2016', '08.2016', '09.2016', '10.2016', '11.2016', '12.2016',
       '01.2017', '02.2017', '03.2017', '04.2017', '05.2017', '06.2017',
       '07.2017', '08.2017', '09.2017', '10.2017', '11.2017', '12.2017',
       '01.2018', '02.2018', '03.2018', '04.2018', '05.2018', '06.2018',
       '07.2018', '08.2018', '09.2018', '10.2018', '11.2018', '12.2018',
       '01.2019', '02.2019', '03.2019', '04.2019', '05.2019', '06.2019',
       '07.2019', '08.2019', '09.2019', '10.2019', 

##df_powód_zamachu

In [ ]:
# Połącz oba DataFrame'y
df_powód_zamachu = pd.concat([df_powód_zamachu_2013_2022, df_powód_zamachu_2023])
df_powód_zamachu.fillna(0, inplace=True)

In [ ]:
df_powód_zamachu.head(1)

,Powód_HIV/AIDS,Powód_Śmierć_bliskiego,Powód_Bezdomność,Powód_Zdrowie,Powód_Praca,Powód_Miłość,Powód_Konflikty_inni,Powód_Utrata_utrzymania,Powód_Choroby_fiz,Powód_Kalectwo,Powód_Inne,Powód_Choroba,Powód_Choroby_psych,Powód_Przestępstwa,Powód_Finanse,Powód_Mobbing,Powód_Ciąża,Powód_Konflikty_rodzinne
0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0


In [ ]:
df_powód_zamachu.shape

(128332, 18)

In [ ]:
df_powód_zamachu_2013_2022.head(1)

,Powód_HIV/AIDS,Powód_Śmierć_bliskiego,Powód_Bezdomność,Powód_Zdrowie,Powód_Praca,Powód_Miłość,Powód_Konflikty_inni,Powód_Utrata_utrzymania,Powód_Choroby_fiz,Powód_Kalectwo,Powód_Inne,Powód_Choroba,Powód_Choroby_psych,Powód_Przestępstwa,Powód_Finanse,Powód_Mobbing,Powód_Ciąża,Powód_Konflikty_rodzinne
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_powód_zamachu_2013_2022.shape

(113196, 18)

In [ ]:
df_powód_zamachu_2023.head(1)

,Powód_HIV/AIDS,Powód_Śmierć_bliskiego,Powód_Bezdomność,Powód_Zdrowie,Powód_Praca,Powód_Miłość,Powód_Konflikty_inni,Powód_Utrata_utrzymania,Powód_Choroby_fiz,Powód_Kalectwo,Powód_Inne,Powód_Finanse,Powód_Choroby_psych,Powód_Przestępstwa,Powód_Mobbing,Powód_Ciąża,Powód_Konflikty_rodzinne
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1


In [ ]:
df_powód_zamachu_2023.shape

(15136, 17)

#Zapis

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# df_raw.to_csv('/content/drive/My Drive/preped_samobojstwa_2013_2022.csv', index=False)
# df_powód_zamachu.to_csv('/content/drive/My Drive/preped_powod_zamachu_samobojstwa_2013_2022.csv', index=False)

In [ ]:
file_name = 'preped_samobojstwa.csv'
df_raw.to_csv(file_name, index=False)
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
file_name = 'preped_powod_zamachu_samobojstwa.csv'
df_powód_zamachu.to_csv(file_name, index=False)
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>